In [1]:
import numpy as np
import pandas as pd
from generate_prob_chart import generate_dest_prob_chart, generate_prob_chart
from numpy.random import choice
from tqdm import tqdm, trange

In [2]:
def cost_info(
        level, event_15=False,
        dest_prevention=False, discount=False,
) -> list:
    result = []
    start_weight = [25]*10 + [400, 220, 150, 110, 75] + [200]*10
    start_weight = list(enumerate(start_weight)) 
    cost = lambda start, weight: round(1000 + level**3 * (start+1)**(1 if start<10 else 2.7) / weight, -2) * \
        (1 - discount*0.3 + (1 if dest_prevention and ((start==15 and not event_15) or start==16) else 0))

    for start, weight in start_weight[:15]:
        result.append(round(cost(start, weight)))
    
    for start, weight in start_weight[15:17]:
        result.append(round(cost(start, weight)))
    
    for start, weight in start_weight[17:]:
        result.append(round(cost(start, weight)))

    return result

In [3]:
def cost_analysis(
        level, starcatch=False, event_15=False,
        dest_prevention=False, discount=False,
        pretty=False,
) -> pd.DataFrame:
    prob = generate_prob_chart(starcatch, event_15, dest_prevention)
    cost_chart = np.zeros((10, 10))  # 15->16 ~ 24->25
    cost = cost_info(level, event_15, dest_prevention, discount)
    cost_one_step = [cost[start]/prob[start, 0] for start in [12, 13, 14]]

    for start in range(15, 25):
        _original_cost = round(1000 + level**3 * (start-1)**2.7 / 200, -2) * (1-discount*0.3)
        _original_cost = round(_original_cost)  # C_{i-2}

        cost_one_step.append(cost[start] + prob[start, 2]*cost[start-1])
        cost_one_step[-1] += cost_one_step[-2]*prob[start, 2]*prob[start-1, 1:3].sum()
        cost_one_step[-1] += _original_cost*prob[start, 2]*prob[start-1, 2]
        cost_one_step[-1] += sum(cost_one_step[:-1]) * (prob[start, 2]*prob[start-1, -1] + prob[start, -1])
        cost_one_step[-1] /= prob[start, 0]
    
    np.fill_diagonal(cost_chart, cost_one_step[3:])
    cost_chart = pd.DataFrame(cost_chart, columns=range(15, 25), index=range(16, 26))
    
    for start in range(15, 24):
        for end in range(start+2, 26):
            cost_chart.loc[end, start] = cost_chart.loc[start+1:end, start:end-1].values.diagonal().sum()

    if pretty is True:
        return cost_chart.loc[:22, :21].map(lambda x: '' if x==0 else f'{x:,.0f}')
    else:
        return cost_chart

In [4]:
cost_analysis(160, False, False, False, False, pretty=True)

,15,16,17,18,19,20,21
16,"154,924,012",,,,,,
17,"692,983,387","538,059,375",,,,,
18,"1,977,820,489","1,822,896,477","1,284,837,102",,,,
19,"4,647,638,916","4,492,714,904","3,954,655,529","2,669,818,427",,,
20,"9,893,731,177","9,738,807,164","9,200,747,789","7,915,910,688","5,246,092,261",,
21,"12,566,598,493","12,411,674,481","11,873,615,106","10,588,778,004","7,918,959,577","2,672,867,316",
22,"21,510,100,547","21,355,176,535","20,817,117,160","19,532,280,058","16,862,461,631","11,616,369,371","8,943,502,054"


# 15 -> 21

In [14]:
def conditional_cost_analysis(level, starcatch=False, event_15=False, dest_prevention=False, discount=False, pretty=False):
    prob = generate_prob_chart(starcatch, event_15, dest_prevention)
    dest = generate_dest_prob_chart(starcatch, event_15, dest_prevention, pretty=False)
    cost = cost_info(160, event_15, dest_prevention, discount)
    temp = [cost[14] / prob[14, 0]]

    for start in range(15, 22):
        _original_cost = round(1000 + level**3 * (start-1)**2.7 / 200, -2) * (1-discount*0.3)
        _original_cost = round(_original_cost)  # C_{i-2}
        dest_1 = dest.loc[start+1, start]  # P(d_i=0)
        dest_2 = dest.loc[start+1, start-1] if start>15 else dest_1  #  P(d_{i-1:i}=0) = P(d_{i-1}=0)P(d_i=0)
        pc = prob[start-1, 0]*dest_1 + prob[start-1, 1:3].sum()*dest_2  # p_{C_i}

        temp.append(cost[start] * (prob[start, 0] + prob[start, 1]*dest_1 + prob[start, 2]*pc))
        temp[-1] += cost[start-1]*prob[start, 2] * pc
        temp[-1] += _original_cost*prob[start, 2]*prob[start-1, 2] * dest_2
        temp[-1] += temp[-2]*prob[start, 2]*prob[start-1, 1:3].sum() * dest_2
        temp[-1] /= (1-prob[start, 1]-prob[start, 2]*prob[start-1, 0])*dest_1 - prob[start, 2]*prob[start-1, 1:3].sum()*dest_2

    cost_chart = np.zeros((7, 7))
    np.fill_diagonal(cost_chart, temp[1:]); del temp
    cost_chart = pd.DataFrame(
        cost_chart,
        columns=range(15, 22),
        index=range(16, 23),
    )

    for start in range(15, 22):
        for end in range(start+2, 23):
            cost_chart.loc[end, start] = cost_chart.loc[start+1:end, start:end-1].values.diagonal().sum()
    
    if pretty is True:
        return cost_chart.map(lambda x: '' if x==0 else f'{x:,.0f}')

In [15]:
conditional_cost_analysis(160, False, False, False, False, pretty=True)

,15,16,17,18,19,20,21
16,"113,752,336",,,,,,
17,"428,986,908","315,234,571",,,,,
18,"927,701,763","813,949,426","498,714,855",,,,
19,"1,484,503,493","1,370,751,156","1,055,516,585","556,801,730",,,
20,"2,012,870,751","1,899,118,414","1,583,883,843","1,085,168,988","528,367,258",,
21,"2,218,519,399","2,104,767,063","1,789,532,491","1,290,817,637","734,015,907","205,648,649",
22,"2,609,617,275","2,495,864,939","2,180,630,368","1,681,915,513","1,125,113,783","596,746,525","391,097,876"


# Simulator

In [6]:
def simulator(level, start, end, starcatch=False, event_15=False, dest_prevention=False, discount=False):
    prob = generate_prob_chart(starcatch, event_15, dest_prevention)
    cost = cost_info(level, event_15, dest_prevention, discount)
    total_cost = 0

    log = [start]*3; log_append = log.append
    current = start

    while current < end:
        total_cost += cost[current]

        if log[-3] - log[-1] == 2:
            current += 1
            log_append(current)
        else:
            current = choice([current+1, current, current-1, -1], p=prob[current, :])
            log_append(current)
            if current == -1:
                return None, log

    log = log[2:]
    return total_cost, log

In [7]:
result = []; result_append = result.append
for _ in trange(100000):
    total_cost, _ = simulator(160, 19, 20)
    if total_cost:
        result_append(total_cost)

100%|██████████| 100000/100000 [00:27<00:00, 3594.08it/s]


In [16]:
print(f'{np.mean(result):,.0f}')

528,407,934
